In [ ]:
import json
import sys
import os
from pymongo import MongoClient
import io
from bson import ObjectId
import wget
import subprocess
import shutil
import nltk
from ast import literal_eval

In [ ]:
client = MongoClient('obfuscated')
db = client['obfuscated']

In [ ]:
def getCategoriesBelowOrEqThresholds(thrsSymbols, thrsInstances):
    hash_map = {}
    years = ["2015C1","2016C1","2017C1","2018C1"]
    
    for year in years:
        collectionYear = db[str(year)]
        mydoc = collectionYear.find({ 'methods_retrieved': True })
        for doc in mydoc:
            word = doc['matching_tag']
            
            for modified in doc['modified_c_files']:
                for method in modified['methods_infile']:
                    
                    old_method_string = method['old_method'].strip()
                    new_method_string = method['new_method'].strip()
                    
                    nltk_tokens_old = nltk.word_tokenize(old_method_string)
                    nltk_tokens_new = nltk.word_tokenize(new_method_string)
                    
                    if len(nltk_tokens_old) <= thrsSymbols and len(nltk_tokens_new) <= thrsSymbols:

                        if word in hash_map:
                            hash_map[word] = hash_map[word] + 1
                        else:
                            hash_map[word] = 1
                    else:
                        continue
                        
    for k,v in sorted(hash_map.items(), key=lambda d_values: d_values[1], reverse=True):
        if v < thrsInstances:
            hash_map.pop(k)
            #print(k, str(v))
    return sorted(hash_map.items(), key=lambda d_values: d_values[1], reverse=True)



In [ ]:
def getStringQuery(my_dict):
    m_dict = my_dict
    str=""
    for k in m_dict:
        str=str+'{"matching_tag": "'+k[0]+'"},'
    return str[:-1]

In [ ]:
def getFilesWithGivenCategories(categories, thrsSymbols):
    years = ["2015C1","2016C1","2017C1","2018C1"]
    
    for year in years:
        collectionYear = db[str(year)]
        
        myQuery = '{"$and":[{ "$or": ['+categories+'] }, {"methods_retrieved": True} ] }'
        myQueryDict = literal_eval(myQuery)
        mydoc = collectionYear.find(myQueryDict)
        with open("matching_tags"+".txt", 'a') as tags:
            with open("vulnerable_methods.txt", 'a') as vuln_met:
                for doc in mydoc:
                    for file in doc["modified_c_files"]:
                        for method in file["methods_infile"]:
                            
                            old_method_string = method['old_method'].strip()
                            new_method_string = method['new_method'].strip()
                    
                            nltk_tokens_old = nltk.word_tokenize(old_method_string)
                            nltk_tokens_new = nltk.word_tokenize(new_method_string)
                    
                            if len(nltk_tokens_old) <= thrsSymbols and len(nltk_tokens_new) <= thrsSymbols:
                                tags.write(doc["matching_tag"])
                                tags.write('\n')
                                vuln_met.write(method["old_method"])
                                vuln_met.write('\n')
    print("The files have been written")

In [ ]:
def getFilesWithGivenCategoriesVulnAndNoVuln(categories, thrsSymbols):
    years = ["2015C1","2016C1","2017C1","2018C1"]
    
    for year in years:
        collectionYear = db[str(year)]
        
        myQuery = '{"$and":[{ "$or": ['+categories+'] }, {"methods_retrieved": True} ] }'
        myQueryDict = literal_eval(myQuery)
        mydoc = collectionYear.find(myQueryDict)
        with open("matching_tags_VNV"+".txt", 'a') as tags:
            with open("vulnerable_methods_VNV.txt", 'a') as vuln_met:
                for doc in mydoc:
                    for file in doc["modified_c_files"]:
                        for method in file["methods_infile"]:
                            
                            old_method_string = method['old_method'].strip()
                            new_method_string = method['new_method'].strip()
                    
                            nltk_tokens_old = nltk.word_tokenize(old_method_string)
                            nltk_tokens_new = nltk.word_tokenize(new_method_string)
                    
                            if len(nltk_tokens_old) <= thrsSymbols and len(nltk_tokens_new) <= thrsSymbols:
                                tags.write("VULN")
                                tags.write('\n')
                                tags.write("NOVULN")
                                tags.write('\n')
                                vuln_met.write(method["old_method"])
                                vuln_met.write('\n')
                                vuln_met.write(method["new_method"])
                                vuln_met.write('\n')
    print("The files have been written")

In [ ]:
def getFilesByCategory(my_dict, thrsSymbols):
    
    years = ["2015C1","2016C1","2017C1","2018C1"]
    
    
    hash_map = {}
    
    for k in my_dict:
        methods = []
        category = k[0]
        print(category)
        current_directory = os.getcwd()
        final_directory = os.path.join(current_directory, category)
        if not os.path.exists(final_directory):
            os.makedirs(final_directory)
          
        for year in years:
            print(year)
            collectionYear = db[str(year)]

            myQuery = '{"$and":[{ "matching_tag": "'+category+'" }, {"methods_retrieved": True} ] }'
            myQueryDict = literal_eval(myQuery)
            mydoc = collectionYear.find(myQueryDict)
            
            with open(category+"/matching_tags_"+category+".txt", 'a') as tags:
                with open(category+"/vulnerable_methods_"+category+".txt", 'a') as vuln_met:
                    for doc in mydoc:
                        word = doc['matching_tag']
                        for file in doc["modified_c_files"]:
                            for method in file["methods_infile"]:

                                old_method_string = method['old_method'].strip()
                                new_method_string = method['new_method'].strip()
                    
                                nltk_tokens_old = nltk.word_tokenize(old_method_string)
                                nltk_tokens_new = nltk.word_tokenize(new_method_string)

                                if len(nltk_tokens_old) <= thrsSymbols and len(nltk_tokens_new) <= thrsSymbols:
                                    
                                    if old_method_string == new_method_string:
                                        continue
                                    else:
                                        if not old_method_string in methods and not new_method_string in methods:
                                            methods.append(old_method_string)
                                            methods.append(new_method_string)
                                            tags.write("VULN")
                                            tags.write('\n')
                                            tags.write("NOVULN")
                                            tags.write('\n')
                                            vuln_met.write(old_method_string)
                                            vuln_met.write('\n')
                                            vuln_met.write(new_method_string)
                                            vuln_met.write('\n')
                                            if word in hash_map:
                                                hash_map[word] = hash_map[word] + 1
                                            else:
                                                hash_map[word] = 1
                                        else:
                                            continue
                                            
        print("Numero total: "+str(len(methods)))
        print("Numero unicos: "+str(len(list(dict.fromkeys(methods)))))
    
    print("The files have been written")
    print(hash_map)

In [ ]:
thrsSymbols = 500
thrsInstances = 10000

In [ ]:
list_categories = getCategoriesBelowOrEqThresholds(thrsSymbols, thrsInstances)
print(list_categories)

In [ ]:
string_categories = getStringQuery(list_categories)
print(string_categories)

In [ ]:
getFilesByCategory(list_categories, thrsSymbols)